In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Data Mining/Information Retrieval

/content/drive/My Drive/Data Mining/Information Retrieval


# get artists from musixmatch

In [ ]:
import requests

fo = open('artists.txt', 'r', encoding='utf-8')
lines = fo.readlines()
fo.close()

singlish_artists = []
for line in lines:
    artist = line.strip()
    singlish_artists.append(artist)

base_url = 'https://api.musixmatch.com/ws/1.1/'
musixmatch_artists = {}

fo = open('musixmatch_artists.csv', 'w', encoding='utf-8')
fo.write('artist_id;artist_name;artist_rating\n')

i = 0
for singlish_artist in singlish_artists:
    print('%i searching artists for %s' % (i, singlish_artist), end=' --> ')
    i += 1
    url = base_url + 'artist.search'
    params = {'apikey': apikey, 'format': 'json', 'q_artist': singlish_artist}
    response = requests.get(url, params=params).json()
    artist_list = response['message']['body']['artist_list']
    j = 0
    for artist in artist_list:
        artist = artist['artist']
        artist_id = artist['artist_id']
        artist_name = artist['artist_name']
        artist_rating = artist['artist_rating']
        artist_credits_artists_list = artist['artist_credits']['artist_list']
        if len(artist_credits_artists_list) > 0:
            artist_list.extend(artist_credits_artists_list)
        if not musixmatch_artists.get(artist_id, False):
            musixmatch_artists[artist_id] = artist_name
            fo.write(str(artist_id) + ';' + artist_name + ';' + str(artist_rating) + '\n')
            j += 1
    print('found %i artists' % j)
fo.close()

print('added %i artists from musixmatch' % len(musixmatch_artists))

FileNotFoundError: ignored

# get tracks from musixmatch

In [ ]:
import requests

fo = open('musixmatch_artists.csv', 'r', encoding='utf-8')
lines = fo.readlines()
fo.close()

musixmatch_artists = {}
for line in lines[1:]:
    artist_id, artist_name, artist_rating = line.strip().split(';')
    musixmatch_artists[artist_id] = {'artist_id': artist_id, 'artist_name': artist_name, 'artist_rating': artist_rating}

base_url = 'https://api.musixmatch.com/ws/1.1/'
musixmatch_tracks = {}

fo = open('musixmatch_tracks.csv', 'w', encoding='utf-8')
fo.write('track_id;track_name;track_rating;has_lyric;album_id;album_name;artist_id;artist_name\n')

i = 0
for musixmatch_artist_id in list(musixmatch_artists.keys()):
    print('%i searching songs for %s' % (i, musixmatch_artists[musixmatch_artist_id]['artist_name']), end=' --> ')
    i += 1
    url = base_url + 'track.search'
    params = {'apikey': apikey, 'format': 'json', 'f_artist_id': musixmatch_artist_id, 'f_has_lyrics': 1, 'page_size': 100, 'page': 1}
    response = requests.get(url, params=params).json()
    track_list = response['message']['body']['track_list']
    j = 0
    for track in track_list:
        track = track['track']
        track_id = track['track_id']
        track_name = track['track_name']
        track_rating = track['track_rating']
        has_lyrics = track['has_lyrics']
        album_id = track['album_id']
        album_name = track['album_name']
        artist_id = track['artist_id']
        artist_name = track['artist_name']
        if not musixmatch_tracks.get(track_id, False):
            musixmatch_tracks[track_id] = track_name
            fo.write(str(track_id) + ';' + track_name + ';' + str(track_rating) + ';' + str(has_lyrics) + ';' + str(album_id) + ';' + album_name + ';' + str(artist_id) + ';' + artist_name + '\n')
            j += 1
    print('found %i songs' % j)
fo.close()

print('added %i songs from musixmatch' % len(musixmatch_tracks))

# get lyrics from musixmatch

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
from selenium import webdriver

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

fo = open('musixmatch_tracks.csv', 'r', encoding='utf-8')
lines = fo.readlines()
fo.close()

base_url = 'https://www.musixmatch.com/lyrics/'

def get_html_source(url):
    browser = webdriver.Chrome('chromedriver', options=chrome_options)
    browser.get(url)
    html_source = browser.page_source
    browser.quit()
    return html_source

def get_elements(html_source):
    elements = []
    if '<span class="lyrics__content__ok">' in html_source:
        elements = html_source.split('<span class="lyrics__content__ok">')
    elif '<span class="lyrics__content__warning">' in html_source:
        elements = html_source.split('<span class="lyrics__content__warning">')
    elif '<span class="lyrics__content__error">' in html_source:
        elements = html_source.split('<span class="lyrics__content__error">')
    return elements

fo = open('musixmatch_lyrics2364-2400.json', 'w', encoding='utf-8')
fo.write('[')
c = 0
for line in lines:
    track_id, track_name, track_rating, has_lyrics, album_id, album_name, artist_id, artist_name = line.strip().split(';')
    artist_name = artist_name.replace('&', '')
    artist_name = artist_name.replace('"', '')
    artist_name = artist_name.replace('.', '')
    param_artist = '-'.join(artist_name.split())
    track_name = track_name.replace('(', '')
    track_name = track_name.replace(')', '')
    track_name = track_name.replace('"', '')
    track_name = track_name.replace('.', '')
    param_track = '-'.join(track_name.split('–')[0].split())
    if has_lyrics:
        print('searching lyrics for %s' % track_name, end=' --> ')
        url = base_url + param_artist + '/' + param_track
        html_source = get_html_source(url)
        elements = get_elements(html_source)
        if len(elements) == 0:
            if 'We detected that your IP is blocked' in html_source:
                print('IP is blocked', url)
                break
            elif 'Think is wrong? You can always add the lyrics' in html_source:
                print('not added yet', url)
                continue
            else:
                url = base_url + param_artist + '-2/' + param_track
                html_source = get_html_source(url)
                elements = get_elements(html_source)
                if len(elements) == 0:
                    if 'We detected that your IP is blocked' in html_source:
                        print('IP is blocked', url)
                        break
                    elif 'Think is wrong? You can always add the lyrics' in html_source:
                        print('not added yet', url)
                        continue
                    else:
                        print('not found on website', url)
                        continue
        lyrics = ''
        for i in range(1, len(elements)):
            element = elements[i].split('</span>')[0]
            lyrics += element
        album_name = album_name.replace('"', '')
        lyrics = lyrics.replace('"', '')
        is_sinhala = False
        for character in lyrics:
            if 3456 <= ord(character) and ord(character) <= 3583:
                is_sinhala = True
                break
        if not is_sinhala:
            print('not Sinhala', url)
            continue
        fo.write('{')
        fo.write('"track_id":"%s",' % str(track_id))
        fo.write('"track_name":"%s",' % track_name)
        fo.write('"track_rating":"%s",' % str(track_rating))
        fo.write('"album_name":"%s",' % album_name)
        fo.write('"artist_name":"%s",' % artist_name)
        fo.write('"lyrics":"%s",' % lyrics)
        fo.write('"url":"%s"' % url)
        fo.write('},')
        c += 1
        print('lyrics added --> count = %i' % c)
fo.write(']')
fo.close()

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
from selenium import webdriver

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

track_ids = []
fo = open('tracks.txt', 'r')
lines = fo.readlines()
for line in lines:
  track_ids.append(line.strip())
fo.close()

fo = open('musixmatch_tracks.csv', 'r', encoding='utf-8')
lines = fo.readlines()
fo.close()

base_url = 'https://www.musixmatch.com/lyrics/'

def get_html_source(url):
    browser = webdriver.Chrome('chromedriver', options=chrome_options)
    browser.get(url)
    html_source = browser.page_source
    browser.quit()
    return html_source

def get_elements(html_source):
    elements = []
    if '<span class="lyrics__content__ok">' in html_source:
        elements = html_source.split('<span class="lyrics__content__ok">')
    elif '<span class="lyrics__content__warning">' in html_source:
        elements = html_source.split('<span class="lyrics__content__warning">')
    elif '<span class="lyrics__content__error">' in html_source:
        elements = html_source.split('<span class="lyrics__content__error">')
    return elements

def all_replace(word, str1, str2):
    while str1 in word:
        word = word.replace(str1, str2) 
    return word

fo = open('musixmatch_lyrics_other3.json', 'w', encoding='utf-8')
fo.write('[')
c = 0
for line in lines[2001:]:
    track_id, track_name, track_rating, has_lyrics, album_id, album_name, artist_id, artist_name = line.strip().split(';')
    if track_id in track_ids:
        continue
    artist_name = all_replace(artist_name, '&', '')
    artist_name = all_replace(artist_name, '-', '')
    artist_name = all_replace(artist_name, '"', '')
    artist_name = all_replace(artist_name, "'", '')
    artist_name = all_replace(artist_name, '.', '-')
    artist_name = all_replace(artist_name, ',', '-')
    param_artist = '-'.join(artist_name.split())
    param_artist = all_replace(param_artist, '--', '-')
    track_name = all_replace(track_name, '&', '')
    track_name = all_replace(track_name, '(', '')
    track_name = all_replace(track_name, ')', '')
    track_name = all_replace(track_name, '[', '')
    track_name = all_replace(track_name, ']', '')
    track_name = all_replace(track_name, '/', '')
    track_name = all_replace(track_name, '"', '')
    track_name = all_replace(track_name, "'", '')
    track_name = all_replace(track_name, '-', '')
    track_name = all_replace(track_name, '.', '-')
    track_name = all_replace(track_name, ',', '-')
    param_track = '-'.join(track_name.split('–')[0].split())
    param_track = all_replace(param_track, '--', '-')
    if has_lyrics:
        print('searching lyrics for %s' % track_name, end=' --> ')
        url = base_url + param_artist + '/' + param_track
        html_source = get_html_source(url)
        elements = get_elements(html_source)
        if len(elements) == 0:
            if 'We detected that your IP is blocked' in html_source:
                print('IP is blocked', url)
                break
            elif 'Think is wrong? You can always add the lyrics' in html_source:
                print('not added yet', url)
                continue
            else:
                url = base_url + param_artist + '-2/' + param_track
                html_source = get_html_source(url)
                elements = get_elements(html_source)
                if len(elements) == 0:
                    if 'We detected that your IP is blocked' in html_source:
                        print('IP is blocked', url)
                        break
                    elif 'Think is wrong? You can always add the lyrics' in html_source:
                        print('not added yet', url)
                        continue
                    else:
                        print('not found on website', url)
                        continue
        lyrics = ''
        for i in range(1, len(elements)):
            element = elements[i].split('</span>')[0]
            lyrics += element
        album_name = album_name.replace('"', '')
        lyrics = lyrics.replace('"', '')
        is_sinhala = False
        for character in lyrics:
            if 3456 <= ord(character) and ord(character) <= 3583:
                is_sinhala = True
                break
        if not is_sinhala:
            print('not Sinhala', url)
            continue
        fo.write('{')
        fo.write('"track_id":"%s",' % str(track_id))
        fo.write('"track_name":"%s",' % track_name)
        fo.write('"track_rating":"%s",' % str(track_rating))
        fo.write('"album_name":"%s",' % album_name)
        fo.write('"artist_name":"%s",' % artist_name)
        fo.write('"lyrics":"%s",' % lyrics)
        fo.write('"url":"%s"' % url)
        fo.write('},')
        c += 1
        print('lyrics added --> count = %i' % c)
fo.write(']')
fo.close()